Step1 : 데이터 수집하기
웹캠에서 얼굴 부분만 검출하여 사진 100장 찍어 폴더에 저장하기

1) cv2.CascadeClassifier() 객체 선언하기
2) cv2.VideoCapture(0) 객체 선언하기
3) 웹캠이 열려있으면
frame 을 읽어서
회색조로 변경
detectMultiScale() 사용하여 얼굴 감지하기
감지된 얼굴에 대해 사이즈  줄여 지정한 폴더에 구분자를 붙여 이미지로 저장하기(cv2.imwrite())
화면에는 카운트 숫자 표기하기
'q'로 종료하거나 count 가 100일때 종료하기

Step2 : 모델 학습하기
저장된 얼굴 이미지를 읽어서 모델 학습하기
1) 저장된 폴더에서 이미지를 읽어(cv2.imread() ) 회색조로 저장하기
2) 100장의 이미지를 리스트에 학습할 데이터(Training_Data) 와 Labels을 append 하여 각각 저장하기
3) 모델 생성하기 : model = cv2.face.LBPHFaceRecognizer_create()
4) 모델 학습하기 : model.train(Training_Data, Labels)
5) 훈련된 모델 파일로 저장하기 :model.write('face_model.yml')

Step3 : 얼굴 인식하기
분류기를 사용하여 웹캠에 비친 얼굴 중 학습한 사용자 인식하기
1) cv2.CascadeClassifier() 객체 선언하기
2) cv2.VideoCapture(0) 객체 선언하기
3) 웹캠이 열려있으면
frame 을 읽어서
회색조로 변경
detectMultiScale() 사용하여 얼굴 감지하기
얼굴이 검출된 경우
회색조로 변경하고
모델 예측하기 : model.predict()
예측 결과 신뢰도로가 75 이상인경우 화면에 "ACCESS GRANTED"
        아니면 "ACCESS DENIED"
'q'로 종료

In [ ]:
import cv2              # OpenCV 라이브러리 가져오기
import numpy as np      # Numpy 라이브러리 가져오기
import sys
import os
from pathlib import Path

# 적용 가능 기술: 특정 색상 셔츠 이용하기, 특정 색상의 종이 이용하기
# 학생들은 모든 가능성을 자유롭게 생각하고 상대 팀이 본인이 무엇을 하려고 하는지 알아채는 것을 어렵게 만들 수 있습니다.
#your code here 

# 전통적인 cv로 만든 classifier임
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) #' 첫 번째' 카메라(웹캠)로 VideoCapture 객체 생성  
saved_count = 0
dir_path = Path("./face")

if(dir_path .exists()):
    pass
else:
    os.mkdir("./face")

while(True):
    ret, frame = camera.read()                              # 프레임 단위로 캡처    
    frame = cv2.resize(frame, (800, 600))
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)    
    faces = face_classifier.detectMultiScale(frame, 1.3, 5)
    
    # 화면에 비친 인원 모두에게 사각형 표시
    # 접근 허가라고 출력
    if(len(faces) > 0): 
        for (x, y, w, h) in faces:
            crop = frame[y:y+h, x:x+w].copy()
            if(saved_count < 100):
                file_path = "./face/" + str(saved_count) + ".jpg"
                cv2.imwrite(file_path, crop)
                saved_count += 1 
                cv2.imshow('Press Spacebar to Exit',crop)              # 프레임 표시
            else:
                camera.release()  
                cv2.destroyAllWindows()
                exit()
    
    if (cv2.waitKey(1) & 0xFF == ord('q') & (saved_count > 100)):  # 스페이스바가 감지되면 중지
        break

camera.release()                           # 스페이스바가 감지된 후 창을 종료
cv2.destroyAllWindows()

In [ ]:
import cv2              # OpenCV 라이브러리 가져오기
import numpy as np      # Numpy 라이브러리 가져오기
import sys
import os
import pandas as pd
from pathlib import Path

Training_Data = []
Labels = []

for i in range(100):
    Training_Data.append(cv2.imread("./face/"  + str(i) + ".jpg", cv2.IMREAD_GRAYSCALE))
    Labels.append(1)

model = cv2.face.LBPHFaceRecognizer_create()

model.train(Training_Data, np.array(Labels))
model.write('face_model.yml')

In [ ]:
import cv2              # OpenCV 라이브러리 가져오기
import numpy as np      # Numpy 라이브러리 가져오기
import sys
import os
from pathlib import Path

# cv2.CascadeClassifier() 
# face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 전통적인 cv로 만든 classifier임
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) #' 첫 번째' 카메라(웹캠)로 VideoCapture 객체 생성  

model=cv2.face.LBPHFaceRecognizer_create();
model.read("face_model.yml")
confidence = 0

while(True):
    ret, frame = camera.read()                              # 프레임 단위로 캡처    
    frame = cv2.resize(frame, (800, 600))
        
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)    
    faces = face_classifier.detectMultiScale(frame, 1.3, 5)
    
    # 화면에 비친 인원 모두에게 사각형 표시
    # 접근 허가라고 출력
    if(len(faces) > 0): 
        for (x, y, w, h) in faces:
            crop = frame[y:y+h, x:x+w].copy()
            
            result = model.predict(crop)
        
            if (result[1] < 50):
                confidence = int(100 * (1 - (result[1] / 300)))
            
                if((result[0] == 1) & (confidence > 75)):
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 100, 100), 5)
                    cv2.putText(frame, "ACCESS GRANTED", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 100, 100),5)

                else:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (100, 100, 255), 5)
                    cv2.putText(frame, "ACCESS DENIED", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (100, 100, 255),5)

    cv2.imshow('Press q to Exit',frame)              # 프레임 표시
    
    if (cv2.waitKey(1) & 0xFF == ord('q')):  # 스페이스바가 감지되면 중지
        break

camera.release()                           # 스페이스바가 감지된 후 창을 종료
cv2.destroyAllWindows()